# Data manipulation and cleaning

In this exercise, we will cover more advanced data manipulation techniques available in the Pandas library, particularly techniques for cleaning up messy data. We will learn to address missing data, recode data columns, and modify data based on conditions. For this exercise, we'll be using the Wake County restaurant available [from the Wake County open data portal](https://data-wake.opendata.arcgis.com/).

## Exercise: import libraries

As before, we need to import the `pandas` library. Go ahead and import that library.

In [ ]:
import pandas as pd

## Read data

There are two data files, one with general restaurant information, and one with inspection results. We need to read both, join them into a single dataset, and do some cleaning.

In [ ]:
restaurants = pd.read_csv("https://raw.githubusercontent.com/mattwigway/odum-intro-python/main/data/restaurants.csv")
inspections = pd.read_csv("https://raw.githubusercontent.com/mattwigway/odum-intro-python/main/data/restaurant_inspections.csv")

## Exercise: Understanding the data

The first step in using a dataset like this is to understand the data. We want to put these datasets together into a single dataset, with one row per restaurant with information on its most recent inspection. A few questions to ask ourselves when looking at this dataset:
- What common columns are there that might allow us to figure out which row in one table is associated with which row in the other table?
- Do both datasets have a single record for each restaurant?

See if you can write Python code to answer these questions.

In [ ]:
restaurants

In [ ]:
inspections

In [ ]:
# we can use .value_counts() to see how many inspection records there are per restaurant
inspections.HSISID.value_counts()

In [ ]:
# it's clear that there is more than one inspection per restaurant, just from the code above.
# But suppose that the restaurants near the start and end of the dataset had one inspection each, but some in the
# middle had multiple - that wouldn't be clear from the preview above. We can look at the .max() function of
# .value_counts() to be sure.

In [ ]:
inspections.HSISID.value_counts().max()

## Finding the most recent inspection of each restaurant

We want one record per restaurant in the inspections dataset, and we want that record to have the most recent inspection. We will first convert the date column to a date type. Then, we will sort the data frame by restaurant ID and date descending, so the most recent inspection is always the first one. Then, we will use a new function of `groupby()`, `nth()`, which returns a new data frame with the n-th row of each group.

### Exercise

Convert the date column to a date type. The codes for specifying date formats [are found here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).

In [ ]:
inspections["DATE_"] = pd.to_datetime(inspections.DATE_, format="%Y/%m/%d %H:%M:%S")

### Sort and group by

We sort the inspections by restaurant ID and date in-place, and then get the first record from each restaurant. Python uses zero-based indexing, so the first element is element 0.

In [ ]:
inspections.sort_values(["HSISID", "DATE_"], ascending=False, inplace=True)

In [ ]:
latest_inspection = inspections.groupby("HSISID").nth(0)
latest_inspection

## Joins

A "join" is a way of putting multiple datasets together when they share a column with a common _key_ (HSISID in this case). There are several types of joins: left joins, right joins, inner joins, and outer joins. They all join two datasets, which are generally referred to as "left" and "right". The result of a join is a new, combined dataset with the columns from both of the original datasets. The types of joins differ in how they handle rows that do not match between the datasets:

- Left join: preserve all rows in the left dataset, drop rows from right dataset that do not match, and leave columns from right dataset empty when there is a left row with no corresponding right row
- Right join: preserve all rows in the right dataset, drop rows from the left that do not match, and leave columns from left dataset empty when there is no corresponding right row
- Inner join: only preserve rows that match between the two datasets
- Outer join: preserve all rows 

Additionally, there are differences in the _number_ of rows that match. A one-to-one join is the simplest, when there is one record for each key in both datasets. Many-to-one and one-to-many joins have duplicated keys in the left or right datasets, respectively, while a many-to-many join has duplicate keys in both datasets. When there are duplicated keys in either dataset, the corresponding rows from the other dataset will be duplicated to match. When there are duplicated keys in _both_ datasets, rows are duplicated so that there is one row with each _combination_ of matching rows - so if there are two rows with the same key in both datasets, there will be four rows in the resulting dataset. We can "validate" our joins to make sure that the rows we think should be unique actually are. If we don't do this, it can lead to misleading results if some records are inadvertently duplicated during the join process.

The default in Pandas is to perform an inner join. Personally, I always prefer to perform a left or outer join, and then drop records that didn't match in a separate step, to make sure that I understand what I'm dropping and don't inadvertently drop records that I didn't mean to. Pandas has the option to create an "indicator" during a join that provides more information about the join.

Confusingly, the function to join two datasets in pandas is called `merge`. There is also a `join` function, which is similar but must work with the index rather than a column of the right dataset, whereas `merge` can use either a column or the index. I recommend always using `merge` rather than `join`. Here we merge on the HSISID in the restaurants dataset. The HSISID column has become the index in latest_inspections, so we either need to specify that we're joining on the index in the right dataset, or reset the index to convert HSISID back to a column.

In [ ]:
restaurant_inspections = restaurants.merge(latest_inspection, how="left", validate="1:1", left_on="HSISID",
                                                right_index=True, indicator=True)

### Investigating the join

We can see that the new data frame has columns from both original data frames. The merge indicator is stored in a new column `_merge`. If contains the values "left_only" or "both" to indicate whether a particular key was only in the left dataset, or in both. It would also contain "right_only" if we'd done an outer join.

In [ ]:
restaurant_inspections.columns

In [ ]:
restaurant_inspections._merge.value_counts()

Most of the restaurants were in both datasets, with a few not having inspection records. We can proceed for now; we'll talk about handling the missing values for those in a few minutes.

## Removing columns

We don't need the `_merge` column any more, so we can drop it. We use the `.drop()` function to do this.

In [ ]:
restaurant_inspections = restaurant_inspections.drop(columns=["_merge"])

### Exercise

Drop other columns we won't need: phone number and state (they're all in North Carolina).

In [ ]:
restaurant_inspections = restaurant_inspections.drop(columns=["PHONENUMBER", "STATE"])

## Doing analysis with the combined dataset

We can now treat this dataset like any other, and use it in analysis.

### Exercise

Compute the median score by city, to see if there are any variations in sanitation between cities (perhaps due to different business licensing regulations, etc.)

In [ ]:
restaurant_inspections.groupby("CITY").SCORE.median()

## Recoding data

Clearly, there are some data issues here, with several cities appearing multiple times with slightly different spellings. We'll use a few string manipulation tools Pandas gives us to try to clean up this data.

Converting names to all caps will help a lot at removing duplicates. Additionally, some of the names do not match even though it looks like they should. This is often due to extra whitespace at the end. We can see if this is the issue by using `.unique()`, which will print out the values with quotes around them to see if there is extra whitespace.

In [ ]:
restaurant_inspections.CITY.unique()

This does appear to be a big part of the issue. We can use the `.str` accessor to access functions for transforming strings. Here, we will use it to convert to upper case, and then strip whitespace from the start and end. Notice that these are chained together - each one returns a series, which has a `.str` accessor of its own.

In [ ]:
restaurant_inspections["clean_city"] = restaurant_inspections.CITY.str.upper().str.strip()

### Exercise

Re-compute the median score, using the `clean_city` column. Are there still any remaining problems with the results?

In [ ]:
restaurant_inspections.groupby("clean_city").SCORE.median()

There are a few remaining issues, which we can fix manually. Morrisville and Holly Springs appear twice due to typos, and Research Triangle Park and Fuquay-Varina have two different spelling. We can use the `.replace` function to replace values with alternate ones.

In [ ]:
restaurant_inspections["clean_city"] = restaurant_inspections.clean_city.replace({
    "HOLLY SPRING": "HOLLY SPRINGS",
    "MORRISVILE": "MORRISVILLE",
    "FUQUAY VARINA": "FUQUAY-VARINA",
    "RTP": "RESEARCH TRIANGLE PARK"
})

### Exercise

Once again recompute the median scores.

In [ ]:
restaurant_inspections.groupby("clean_city").SCORE.median()

## Missing data

Missing data is a constant problem when working with real-world datasets. In Pandas, missing data is generally represented as `np.nan` (not a number) or `None` (a special Python data representing nothing; similar to NULL in other languages). Unlike some languages, many operations in Python will silently ignore missing data (for instance, the median above). We know there is some missing data in this dataset, because some restaurants did not have matching inspections. We can use the `isnull()` function to count how many there are.

In [ ]:
restaurant_inspections.SCORE.isnull().sum()

That's not very many. Let's look at where they are.

This is introducting a new syntax: apply with groupby, which allows performing arbitrary operations on each group. `lambda x:` creates an _anonymous function_  of x(just like the functions we've been using so far, but without a name), and whatever comes after it will be executed for each group. `x` will contain the scores for each group.

If there are a lot of groups, this can be slow, but that's not an issue here.

In [ ]:
restaurant_inspections.groupby("clean_city").SCORE.apply(lambda x: x.isnull().sum())

There are a lot of missing values in Raleigh, but there are also a lot of restaurants in Raleigh. We can replace `.sum()` with `.mean()` to get a proportion.

In [ ]:
restaurant_inspections.groupby("clean_city").SCORE.apply(lambda x: x.isnull().mean())

## Dropping missing data

A relatively small number of data points are missing in any city. We can just drop (delete) the rows with missing data, using the `.dropna()` function. We use the subset argument to only drop rows with a missing score. We don't want to drop restaurants with a missing address line 2, for example.

In [ ]:
restaurant_inspections = restaurant_inspections.dropna(subset=["SCORE"])

And we can then re-run the analysis to confirm there are no more missing values.

In [ ]:
restaurant_inspections.SCORE.isnull().sum()

## Pitfalls of filtered data

When you filter data, e.g. using dropna or the filtering methods discussed in exercise 1, the original index values are preserved even after filtering. For a data frame that was indexed by a sequential row number, this can be quite confusing, as there are now numbers missing. For instance, there is no longer a row with index value 32. This isn't actually a problem, and can be useful in some cases, but it can also be quite confusing. You can use `.reset_index()` to get back to a sequential integer index, if you like. `drop=True` discards the existing index, rather than converting it to a column.

In [ ]:
restaurant_inspections.loc[32]

In [ ]:
restaurant_inspections.reset_index(drop=True, inplace=True)

## More advanced filtering and data selection

The Pandas index can be quite useful, particularly when it is not just a list of numbers, but something meaningful (e.g. the business ID). We can set the index with `set_index`. We've already seen `.loc` used for filtering, but it can also be used for selecting by index. Let's set the index for restaurant inspections to the HSISID, and then look up a restaurant by ID.

In [ ]:
restaurant_inspections = restaurant_inspections.set_index("HSISID")
restaurant_inspections

In [ ]:
restaurant_inspections.loc[4092018099]

You can also select specific columns with `.loc`, by including them after a comma.

In [ ]:
restaurant_inspections.loc[4092018099, "NAME"]

## Modifying data using indexing

Sometimes, you will want to modify data in ways that cannot be directly supported by `.replace` etc. In this case, you can assignment and indexing to change values. Suppose that McDonald's has implemented a corporate program to increase sanitation at their lower-performing franchises, and the program will guarantee that every McDonald's scores at least 98 on their inspections. We'll modify the data so that all McDonald's that scored below 98 have a score of 98, and see how that influences the median scores.

The first step is to find all of the McDonald's restaurants in the data frame. We can use another `.str` function, `.contains()`, to search for text in a string. `.contains` uses [regular expressions](https://docs.python.org/3/library/re.html#regular-expression-syntax). Some special characters have special meanings. For now, we'll just use it to search for the text "McDonald" in the dataset - with no special characters. Since there may be differences in capitalization, we convert the names to uppercase before searching. Whenever searching for a string, I like to first display the matched values to ensure I haven't over-selected (e.g. selecting "Ronald McDonald House Cafeteria" or something).

In [ ]:
restaurant_inspections.loc[restaurant_inspections.NAME.str.upper().str.contains("MCDONALD"), "NAME"]

These all look like actual McDonald's. Next, we can filter by score, and assign a higher score to those that need it.

In [ ]:
restaurant_inspections.loc[
    # we don't need parentheses around this first condition, because it's a single function call, and doesn't use any
    # operators such as ==. If that's hard to remember, you can just put parentheses around each condition - it won't hurt
    # anything to add the extra parentheses
    restaurant_inspections.NAME.str.upper().str.contains("MCDONALD") &
    (restaurant_inspections.SCORE < 98),
    "SCORE"
] = 98

## Exercise

Let's check our work. Use the indexing above to display the scores for all McDonald's restaurants.

In [ ]:
restaurant_inspections.loc[restaurant_inspections.NAME.str.upper().str.contains("MCDONALD"), "SCORE"]

### Exercise

Re-run the per-city median. Did the results change? Why or why not?

In [ ]:
restaurant_inspections.groupby("clean_city").SCORE.median()

### Pitfall: chained indexing

When doing assignments, you need to avoid something called "chained indexing". This is where you use multiple indexing or data selection operations to select the rows and columns you want to assign. Run the following code, and notice you get a warning.

In [ ]:
restaurant_inspections.loc[restaurant_inspections.clean_city=="MORRISVILLE"]["SCORE"]=100

Now, inspect the data frame to see if your assignment worked.

In [ ]:
restaurant_inspections.loc[restaurant_inspections.clean_city == "MORRISVILLE", "SCORE"]

It didn't work. What happened here? This seems similar enough to what was done above.

What happened is that when you select data with Pandas, sometimes you get a "view" of the original data - i.e. you're just looking at a portion of the original data frame, but it's still the same data. Other times, you get a "copy" - the values are the same, but it's a different dataset, so modifying it won't affect the original data.

Pandas executed the code above from left to right. First, it selected all of the restaurants in Morrisville

```python
restaurant_inspections.loc[restaurant_inspections.clean_city == "MORRISVILLE"]
```

This created a copy of the data. When you then assigned a value to the score column of that copy, the original dataset was not updated. This is why we got a SettingWithCopyWarning. You shouldn't rely on this warning, though; sometimes Python will suppress warnings if you have seen them frequently.

### Exercise

How would you set the score for all Morrisville restaurants to 100, without using chained indexing?

In [ ]:
restaurant_inspections.loc[restaurant_inspections.clean_city == "MORRISVILLE", "SCORE"] = 100

# check our work
restaurant_inspections.loc[restaurant_inspections.clean_city == "MORRISVILLE", "SCORE"]

## Wide and long format data

Another common data manipulation task is converting data between "wide" and "long" format. In "wide" format, you will have one row per observation, with many columns, whereas in "long" format you will have multiple rows with fewer columns. For instance, let's compute the median inspection score for all inspections (not just the latest) by city and year.

In [ ]:
inspections["year"] = inspections.DATE_.dt.year

### Exercise

We don't have a `clean_city` variable in the inspections data frame. Merge in the restaurants dataset (this is similar to what we did before, but you want to keep every inspection record), and use the same data cleaning we used before to create the `clean_city` variable. Store the results in a new data frame `by_city_and_year`.

In [ ]:
inspections_with_city = inspections.merge(restaurants, on="HSISID", how="left", validate="m:1", indicator=True)
inspections_with_city._merge.value_counts()

In [ ]:
inspections_with_city.dropna(subset=["_merge"], inplace=True)

In [ ]:
inspections_with_city["clean_city"] = (
    inspections_with_city.CITY.str.upper().str.strip()
    .replace({
    "HOLLY SPRING": "HOLLY SPRINGS",
    "MORRISVILE": "MORRISVILLE",
    "FUQUAY VARINA": "FUQUAY-VARINA",
    "RTP": "RESEARCH TRIANGLE PARK"
}))

Next, we can group by city and year, and take the median score.

In [ ]:
by_city_and_year = inspections_with_city.groupby(["clean_city", "year"]).SCORE.median()
by_city_and_year

Is this wide or long format?

### Converting long to wide

We can convert long to wide by using the `.unstack` function. This only works when you have multi-column index (in this case, clean_city and year). The `unstack` function will turn the last level of the index into columns.

In [ ]:
by_city_and_year = by_city_and_year.unstack()
by_city_and_year

We can convert back to the original format using the `.stack` function, which converts columns into a hierarchical row index level.

In [ ]:
by_city_and_year.stack()

### Converting a table without a hierarchical index

In the example above, we had a hierarchical index as a result of the groupby, so unstack() made it very simple to convert one index level into column names. If you don't have data indexed by the variables you need to unstack by, you can either 1. set the index, using `.set_index([col1, col2])` where `col2` is the column you want to have become the column names, or use the `.pivot` function. `pivot` takes three named arguments: index, columns, and values, which correspond with the columns to get the index labels, the column names, and the data values from.

In [ ]:
# first, create a long table without a meaningful index
by_city_and_year = inspections_with_city.groupby(["clean_city", "year"]).SCORE.median().reset_index()
by_city_and_year

In [ ]:
# now, pivot
by_city_and_year = by_city_and_year.pivot(index="clean_city", columns="year", values="SCORE")
by_city_and_year

### Exercise

Compute median score by inspector and year, and present it in a table with years for columns. You can use `.unstack` or `.pivot`.

In [ ]:
inspections_with_city.groupby(["INSPECTOR", "year"]).SCORE.median().unstack()